In [ ]:
!pip install keras-tuner --upgrade

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

In [ ]:
# Importing the necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from pylab import rcParams
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)
plt.rcParams['figure.figsize'] = [20, 6]

import random

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Importing the dataset
df= pd.read_csv('dataset.csv', encoding='latin1')
df

In [ ]:
df.info()

In [ ]:
df = df.drop(['numer_sta', 'lat', 'lon', 'storm name', 'Storm'], axis=1)
df.head()

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df = df.set_index("date")

In [ ]:
# Dealing with missing values
print(df.isnull().sum())

In [ ]:
from sklearn.impute import SimpleImputer

# Create an imputer object using median as the strategy
imputer = SimpleImputer(missing_values=np.nan, strategy='median')

# Define the columns where you want to apply the imputation
columns_to_impute = ['Temperature',	'Humidity',	'Wind speed',	'Pressure', 'Wave height', 'Wave period']

# Apply the imputer to the selected columns of the DataFrame
df[columns_to_impute] = imputer.fit_transform(df[columns_to_impute])

# Checking the DataFrame to ensure no more missing values
df.isnull().values.any()

In [ ]:
num_variables = len(df.columns)
fig, axes = plt.subplots(nrows=num_variables, ncols=1, figsize=(20, 5 * num_variables))

# Plot each variable on its own subplot
for i, column in enumerate(df.columns):
    axes[i].plot(df.index, df[column], label=f'{column}', color=plt.cm.tab10(i))
    axes[i].set_title(f'Time Series of {column}', fontsize=16)
    axes[i].legend(loc='upper right')
    axes[i].set_ylabel(column)
    axes[i].tick_params(labelsize=12)

# Set common labels
plt.xlabel("Index (Time)", fontsize=14)
plt.xticks(fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

wave_height = df['Wave height']

# Plot the ACF
plt.figure(figsize=(14,7))
plot_acf(wave_height, lags=100, alpha=0.05)
plt.title('Autocorrelation of Wave Height')
plt.show()

wave_period = df['Wave period']

# Plot the ACF
plt.figure(figsize=(14,7))
plot_acf(wave_period, lags=100, alpha=0.05)
plt.title('Autocorrelation of Wave Period')
plt.show()


Wind_speed = df['Wind speed']

# Plot the ACF
plt.figure(figsize=(14,7))
plot_acf(Wind_speed, lags=100, alpha=0.05)
plt.title('Autocorrelation of Wind Speed')
plt.show()

temperature = df['Temperature']
# Plot the ACF
plt.figure(figsize=(14,7))
plot_acf(temperature, lags=100, alpha=0.05)
plt.title('Autocorrelation of Temperature')
plt.show()

humidity = df['Humidity']
# Plot the ACF
plt.figure(figsize=(14,7))
plot_acf(humidity, lags=100, alpha=0.05)
plt.title('Autocorrelation of Humidity')
plt.show()

pressure = df['Pressure']
# Plot the ACF
plt.figure(figsize=(14,7))
plot_acf(pressure, lags=100, alpha=0.05)
plt.title('Autocorrelation of Pressure')
plt.show()

In [ ]:
# Data Formating/ Preparing the input shape

def df_to_X_y(df_1, window_size=30):
  df_as_np = df_1.to_numpy()
  X = []
  y = []
  for i in range(len(df_as_np)-window_size):
    row = [r for r in df_as_np[i:i+window_size]]
    X.append(row)
    label = [df_as_np[i+window_size][0], df_as_np[i+window_size][1], df_as_np[i+window_size][2], df_as_np[i+window_size][3], df_as_np[i+window_size][4], df_as_np[i+window_size][5]]
    y.append(label)
  return np.array(X), np.array(y)

In [ ]:
X, y = df_to_X_y(df)
print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")

In [ ]:
# Define the size of the training set
train_size = int(len(X) * 0.80)  # Using 80% of data for training and 20% for testing

# Split the data into training and test sets by slicing
X_train, y_train = X[:train_size], y[:train_size]
X_test, y_test = X[train_size:], y[train_size:]

# Print the shapes to verify the split
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

In [ ]:
# Normalization
temp_training_mean = np.mean(X_train[:, :, 0])
temp_training_std = np.std(X_train[:, :, 0])

hum_training_mean = np.mean(X_train[:, :, 1])
hum_training_std = np.std(X_train[:, :, 1])

wind_training_mean = np.mean(X_train[:, :, 2])
wind_training_std = np.std(X_train[:, :, 2])

pres_training_mean = np.mean(X_train[:, :, 3])
pres_training_std = np.std(X_train[:, :, 3])

waveH_training_mean = np.mean(X_train[:, :, 4])
waveH_training_std = np.std(X_train[:, :, 4])

waveP_training_mean = np.mean(X_train[:, :, 5])
waveP_training_std = np.std(X_train[:, :, 5])

def preprocess(X):
  X[:, :, 0] = (X[:, :, 0] - temp_training_mean) / temp_training_std
  X[:, :, 1] = (X[:, :, 1] - hum_training_mean) / hum_training_std
  X[:, :, 2] = (X[:, :, 2] - wind_training_mean) / wind_training_std
  X[:, :, 3] = (X[:, :, 3] - pres_training_mean) / pres_training_std
  X[:, :, 4] = (X[:, :, 4] - waveH_training_mean) / waveH_training_std
  X[:, :, 5] = (X[:, :, 5] - waveP_training_mean) / waveP_training_std

def preprocess_output(y):
  y[:, 0] = (y[:, 0] - temp_training_mean) / temp_training_std
  y[:, 1] = (y[:, 1] - hum_training_mean) / hum_training_std
  y[:, 2] = (y[:, 2] - wind_training_mean) / wind_training_std
  y[:, 3] = (y[:, 3] - pres_training_mean) / pres_training_std
  y[:, 4] = (y[:, 4] - waveH_training_mean) / waveH_training_std
  y[:, 5] = (y[:, 5] - waveP_training_mean) / waveP_training_std
  return y

In [ ]:
preprocess(X_train)
preprocess(X_test)

In [ ]:
preprocess_output(y_train)
preprocess_output(y_test)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner import BayesianOptimization, Objective
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner as kt
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import TimeSeriesSplit



class MyHyperModel(kt.HyperModel):
    def build(self, hp):
        model = keras.Sequential()
        model.add(LSTM(hp.Int('input_unit',min_value=32,max_value=512,step=32),return_sequences=True, input_shape=(30,6)))
        for i in range(hp.Int('n_layers', 1, 4)):
          model.add(LSTM(hp.Int(f'lstm_{i}_units',min_value=32,max_value=512,step=32),return_sequences=True))
        model.add(LSTM(hp.Int('layer_2_neurons',min_value=32,max_value=512,step=32)))
        model.add(Dropout(hp.Float('Dropout_rate',min_value=0,max_value=0.5,step=0.1)))
        model.add(Dense(6, activation='linear'))
        model.compile(
            optimizer=keras.optimizers.Adam(
                hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
            loss='mean_absolute_error',
            metrics=['mean_absolute_error'])
        return model

    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            batch_size=hp.Choice("batch_size", [16, 32, 64, 128]),
            **kwargs,
        )

tuner = kt.BayesianOptimization(
    MyHyperModel(),
    objective="val_mean_absolute_error",
    max_trials=10,
    executions_per_trial=2,
    overwrite=True,
    directory="my_dir",
    project_name="tune_hypermodel",
)

early_stopping_monitor = EarlyStopping(
    monitor='val_mean_absolute_error',
    patience=40,
    min_delta=0.001,
    restore_best_weights=True,
    verbose=1
)


def cross_validate(X, y, n_splits=3):
    tscv = TimeSeriesSplit(n_splits=n_splits)
    for train_index, val_index in tscv.split(X):
        X_train_fold, X_val_fold = X[train_index], X[val_index]
        y_train_fold, y_val_fold = y[train_index], y[val_index]
        tuner.search(
            X_train_fold, y_train_fold, epochs=100,
            validation_data=(X_val_fold, y_val_fold),
            callbacks=[early_stopping_monitor]
        )


# Call to perform cross-validation
cross_validate(X_train, y_train, n_splits=3)

In [ ]:
# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Print the best hyperparameters
print("The best hyperparameters are:")
print(f"Input LSTM units: {best_hps.get('input_unit')}")
print(f"Number of LSTM layers: {best_hps.get('n_layers') + 2}")  # includes input and final LSTM layers
for i in range(best_hps.get('n_layers')):
    print(f"LSTM units in layer {i+1}: {best_hps.get(f'lstm_{i}_units')}")
print(f"Final LSTM layer units: {best_hps.get('layer_2_neurons')}")
print(f"Dropout rate: {best_hps.get('Dropout_rate')}")
print(f"Learning rate: {best_hps.get('learning_rate')}")
print(f"Batch size: {best_hps.get('batch_size')}")

In [ ]:
# Build the model with the best hyperparameters
model = tuner.hypermodel.build(best_hps)

# Training the model with the full training dataset
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=best_hps.get('batch_size'),
    callbacks=[early_stopping_monitor]
)

In [ ]:
# Predicting on unseen data
predictions = model.predict(X_test)

In [ ]:
temp_preds, hum_preds, wind_preds, pres_preds, waveH_preds, waveP_preds = predictions[:, 0], predictions[:, 1], predictions[:, 2], predictions[:, 3], predictions[:, 4], predictions[:, 5]
temp_actuals, hum_actuals, wind_actuals, pres_actuals, waveH_actuals, waveP_actuals = y_test[:, 0], y_test[:, 1], y_test[:, 2], y_test[:, 3], y_test[:, 4], y_test[:, 5]

df_prediction = pd.DataFrame(data={'Temperature Predictions': temp_preds, 'Temperature Actuals':temp_actuals,
                                   'Humidity Predictions': hum_preds, 'Humidity Actuals':hum_actuals,
                                   'Wind speed Predictions': wind_preds, 'Wind speed Actuals': wind_actuals,
                                   'Pressure Predictions': pres_preds, 'Pressure Actuals': pres_actuals,
                                   'Wave height Predictions': waveH_preds, 'Wave height Actuals': waveH_actuals,
                                   'Wave period Predictions': waveP_preds, 'Wave period Actuals': waveP_actuals,

                          })

In [ ]:
df_prediction.head()

In [ ]:
def postprocess_temp(arr):
  arr = (arr*temp_training_std) + temp_training_mean
  return arr

def postprocess_hum(arr):
  arr = (arr*hum_training_std) + hum_training_mean
  return arr

def postprocess_wind(arr):
  arr = (arr*wind_training_std) + wind_training_mean
  return arr

def postprocess_pres(arr):
  arr = (arr*pres_training_std) + pres_training_mean
  return arr

def postprocess_waveH(arr):
  arr = (arr*waveH_training_std) + waveH_training_mean
  return arr

def postprocess_waveP(arr):
  arr = (arr*waveP_training_std) + waveP_training_mean
  return arr

In [ ]:
temp_preds1, hum_preds1, wind_preds1, pres_preds1, waveH_preds1, waveP_preds1  = postprocess_temp(predictions[:, 0]), postprocess_hum(predictions[:, 1]), postprocess_wind(predictions[:, 2]), postprocess_pres(predictions[:, 3]), postprocess_waveH(predictions[:, 4]), postprocess_waveP(predictions[:, 5])

temp_actuals1, hum_actuals1, wind_actuals1, pres_actuals1, waveH_actuals1, waveP_actuals1 = postprocess_temp(y_test[:, 0]), postprocess_hum(y_test[:, 1]), postprocess_wind(y_test[:, 2]), postprocess_pres(y_test[:, 3]), postprocess_waveH(y_test[:, 4]), postprocess_waveP(y_test[:, 5])


df_prediction1 = pd.DataFrame(data={'Temperature Predictions': temp_preds1, 'Temperature Actuals':temp_actuals1,
                                   'Humidity Predictions': hum_preds1, 'Humidity Actuals':hum_actuals1,
                                   'Wind speed Predictions': wind_preds1, 'Wind speed Actuals': wind_actuals1,
                                   'Pressure Predictions': pres_preds1, 'Pressure Actuals': pres_actuals1,
                                   'Wave height Predictions': waveH_preds1, 'Wave height Actuals': waveH_actuals1,
                                    'Wave period Predictions': waveP_preds1, 'Wave period Actuals': waveP_actuals1
                          })

In [ ]:
df_prediction1

In [ ]:
# Temperature prediction
plt.figure(figsize=(20, 8))
plt.plot(df[7311:].index, df_prediction1['Temperature Actuals'], label="Actual Temperature")
plt.plot(df[7311:].index, df_prediction1['Temperature Predictions'], label="Prediction")
plt.legend(loc='best', fontsize='large')
plt.xticks(fontsize=18)
plt.yticks(fontsize=16)
plt.xlabel("Date time")
plt.ylabel("Temperature")
plt.show()

In [ ]:
# Wave height prediction
plt.figure(figsize=(20, 8))
plt.plot(df[7311:].index, df_prediction1['Wave height Actuals'], label="Actual Wave height")
plt.plot(df[7311:].index, df_prediction1['Wave height Predictions'], label="Prediction")
plt.legend(loc='best', fontsize='large')
plt.xticks(fontsize=18)
plt.yticks(fontsize=16)
plt.xlabel("Date time")
plt.ylabel("Wave height")
plt.show()

In [ ]:
# Waind speed prediction
plt.figure(figsize=(20, 8))
plt.plot(df[7311:].index, df_prediction1['Wind speed Actuals'], label="Actual Wind speed")
plt.plot(df[7311:].index, df_prediction1['Wind speed Predictions'], label="Prediction")
plt.legend(loc='best', fontsize='large')
plt.xticks(fontsize=18)
plt.yticks(fontsize=16)
plt.xlabel("Date time")
plt.ylabel("Wind speed")
plt.show()

In [ ]:
# Pressure prediction
plt.figure(figsize=(20, 8))
plt.plot(df[7311:].index, df_prediction1['Pressure Actuals'], label="Actual Pressure")
plt.plot(df[7311:].index, df_prediction1['Pressure Predictions'], label="Prediction")
plt.legend(loc='best', fontsize='large')
plt.xticks(fontsize=18)
plt.yticks(fontsize=16)
plt.xlabel("Date time")
plt.ylabel("Pressure")
plt.show()

In [ ]:
# Humidity prediction
plt.figure(figsize=(20, 8))
plt.plot(df[7311:].index, df_prediction1['Humidity Actuals'], label="Actual Humidity")
plt.plot(df[7311:].index, df_prediction1['Humidity Predictions'], label="Prediction")
plt.legend(loc='best', fontsize='large')
plt.xticks(fontsize=18)
plt.yticks(fontsize=16)
plt.xlabel("Date time")
plt.ylabel("Humidity")
plt.show()

In [ ]:
# Wave period prediction
plt.figure(figsize=(20, 8))
plt.plot(df[7311:].index, df_prediction1['Wave period Actuals'], label="Actual Wave period")
plt.plot(df[7311:].index, df_prediction1['Wave period Predictions'], label="Prediction")
plt.legend(loc='best', fontsize='large')
plt.xticks(fontsize=18)
plt.yticks(fontsize=16)
plt.xlabel("Date time")
plt.ylabel("Wave period")
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy import stats

# Function to calculate Mean Absolute Percentage Error
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Calculate performance metrics
def calculate_metrics(y_true, y_pred, variable_name):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    correlation, p_value = stats.pearsonr(y_true, y_pred)

    print(f'{variable_name} Metrics:')
    print(f'Mean Absolute Error: {mae:.4f}')
    print(f'Mean Squared Error: {mse:.4f}')
    print(f'Root Mean Squared Error: {rmse:.4f}')
    print(f'Mean Absolute Percentage Error: {mape:.2f}%')
    print(f'R2 Score (Coefficient of Determination): {r2:.4f}')
    print(f'Pearson Correlation Coefficient: {correlation:.4f}')
    print(f'P-value of Correlation Coefficient: {p_value:.4g}')
    print("\n")

# Extract the relevant columns
predicted_temperature = df_prediction1['Temperature Predictions']
actual_temperature = df_prediction1['Temperature Actuals']

predicted_humidity = df_prediction1['Humidity Predictions']
actual_humidity = df_prediction1['Humidity Actuals']

predicted_wind_speed = df_prediction1['Wind speed Predictions']
actual_wind_speed = df_prediction1['Wind speed Actuals']

predicted_pressure = df_prediction1['Pressure Predictions']
actual_pressure = df_prediction1['Pressure Actuals']

predicted_wave_height = df_prediction1['Wave height Predictions']
actual_wave_height = df_prediction1['Wave height Actuals']

predicted_period = df_prediction1['Wave period Predictions']
actual_period = df_prediction1['Wave period Actuals']

# Calculate metrics for temperature
calculate_metrics(actual_temperature, predicted_temperature, 'Temperature')

# Calculate metrics for Humidity
calculate_metrics(actual_humidity, predicted_humidity, 'Humidity')

# Calculate metrics for Wind speed
calculate_metrics(actual_wind_speed, predicted_wind_speed, 'Wind Speed')

# Calculate metrics for Pressure
calculate_metrics(actual_pressure, predicted_pressure, 'Pressure')

# Calculate metrics for Wave height
calculate_metrics(actual_wave_height, predicted_wave_height, 'Wave Height')

# Calculate metrics for Wave period
calculate_metrics(actual_period, predicted_period, 'Wave Period')